# Data Generator
Where datasets are large, data is read into the memory in batches, instead of loading the full dataset into the memory. Data Generator assists in this process. In addition, it may also perform necessary data preprocessing operations.
This lab illustrates the concept of Data Generator on CSV files.

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf

In [ ]:
keras.backend.clear_session()

In [ ]:
train = pd.read_csv ('/content/drive/My Drive/Colab Notebooks/Data/online shop_train.csv')
test = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Data/online shop_test.csv')


### To run Data Generator, all dataframes are to be converted to numpy arrays. data is also converted to float.

In [ ]:
X_train = (train.iloc[:,:-1].values).astype('float32')
y_train = (train.iloc[:,-1].values).astype('float32')
X_test = (test.iloc[:,:-1].values).astype('float32')
y_test =(test.iloc[:,-1].values).astype('float32')

In [ ]:
import tensorflow as tf
import keras

In [ ]:
from keras import utils as np_utils
y_tra = keras.utils.to_categorical(y_train, num_classes=None, dtype="float32")
y_tes =  keras.utils.to_categorical(y_test, num_classes=None, dtype="float32")

# Model


In [ ]:
import keras

In [ ]:
model = keras.models.Sequential()
model.add(keras.layers.Dense(512, kernel_initializer = 'he_uniform', activation="relu", input_dim = 12))
model.add(keras.layers.Dense(128, kernel_initializer = 'he_uniform', activation="relu"))
model.add(keras.layers.Dense(96, kernel_initializer = 'he_uniform', activation="relu"))
model.add(keras.layers.Dense(2,kernel_initializer = 'he_uniform', activation="softmax"))

In [ ]:
model.compile (loss = "categorical_crossentropy", 
               optimizer = "adam",
               metrics = ["accuracy"])

In [ ]:
# Create a function that returns an infinitely looping generator - to feed data continuously for all epochs
# permutation features performs shuffling

def get_generator_cyclic(features, labels, batch_size = 1):
  while True:
    for n in range (int(len(features)/batch_size)):
      yield (features[n*batch_size: (n+1)*batch_size], labels[n*batch_size: (n+1)*batch_size])
      permuted = np.random.permutation(len(features))
      features = features[permuted]
      labels = labels[permuted]

In [ ]:
batch_size = 30

In [ ]:
# Generate a cyclic train generator
train_generator_cyclic = (get_generator_cyclic(X_train, y_tra, batch_size=batch_size))

In [ ]:
# Generate a cyclic validation generator

validation_generator_cyclic = (get_generator_cyclic(X_test, y_tes, batch_size=batch_size))

In [ ]:
history = model.fit(train_generator_cyclic, steps_per_epoch=(len(X_train) // batch_size),
                    validation_data = validation_generator_cyclic, validation_steps = (len(X_test) // batch_size), epochs =30)

In [ ]:
import matplotlib.pyplot as plt

pd.DataFrame(history.history).plot(figsize=(8, 5))
plt.grid(True)
plt.gca().set_ylim(0, 1)
plt.show()